## файл для подключения БД 



In [4]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

In [ ]:
### создаем подключение 
SQLALCHEMY_DATABASE_URL = "" # строка подключения к бд

engine = create_engine(SQLALCHEMY_DATABASE_URL) # прослойка к БД
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine) # создатель сессии

Base = declarative_base() # базовый класс - схема объекта

## защита пароля

In [ ]:
# переменные окружения
import os


# bash way (not the best)________
# export POSTGRES_USER=irina
# export POSTGRES_PASSWORD=011192543
# export ...


# file .env
POSTGRES_USER=myuser
POSTGRES_PASSWORD=mypassword
POSTGRES_HOST=localhost
POSTGRES_PORT=5432
POSTGRES_DATABASE=mydatabase

#bash: `source .env`

# .env в коде:
# install python-dotenv
import os
from dotenv import load_dotenv

# Загружаем переменные окружения из .env
load_dotenv()

# Читаем их
user = os.getenv("POSTGRES_USER")
password = os.getenv("POSTGRES_PASSWORD")
host = os.getenv("POSTGRES_HOST")
port = os.getenv("POSTGRES_PORT")
database = os.getenv("POSTGRES_DATABASE")

SQLALCHEMY_DATABASE_URL = f"postgresql://{user}:{password}@{host}:{port}/{database}"


## файл с моделями



### создаем сущности

In [ ]:
# схема отношения юзер
class User(Base):
    __tabelname__ = "user"
    __table.args__ = {'scema': 'cd'}
    id = Column(Integer, primery_key=True, name='')
    name = Column(String)
    surname = Column(String)
    age = Column(Integer)
    
# при запуске файла напрямую (а не импорт): запускается от имени main
if __name__ == '__main__':
    # создание таблиц (еще нет в бд)
    Base.metadata.create_all(engine)
    




## скрипт 
в директории с модулями добавить файл `__init__.py`

### импорт модулей

In [ ]:
# добавить корень с модулями в переменные окружения PYTHONPTH
export PYTHONPTH=$PYTHONPTH:$PWD # терминал

### добавляем запись / сущность в бд

In [ ]:
from <dir>.models import User, SessionLocal


if __name__ == '__main__':
    
    user = User(name='', surname='', age='') # создали сущность
    
    session = SessionLocal() # создание сeccии
    session.add(user) # добавление операции в транзакцию - добавление сущности в БД (после завершения транзакции)
    session.commit() # завершение транзакции, выполнение операций
    

In [ ]:
# c обработкой ошибок

from <dir>.models import User, SessionLocal

if __name__ == '__main__':
    session = SessionLocal()  # Создаем сессию
    try:
        user = User(name='Иван', surname='Иванов', age=30)  # Создаем нового пользователя
        session.add(user)  # Добавляем в сессию
        session.commit()  # Фиксируем изменения в БД
        print("✅ Пользователь успешно добавлен!")
    except Exception as e:
        session.rollback()  # Откатываем транзакцию, если была ошибка
        print(f"❌ Ошибка: {e}")
    finally:
        session.close()  # Закрываем сессию


In [ ]:
# Чтобы продолжать работать с объектом, его надо обновить
session.refresh(user)  # ✅ Теперь объект снова связан с сессией


# Лучше работать через контекстный менеджер
with SessionLocal() as session:
    user = session.query(User).filter_by(name="Иван").first()
    user.age = 31
    session.commit()  # ✅ Коммитим изменения


In [ ]:
# dependency injection

from fastapi import FastAPI, Depends, HTTPException
from sqlalchemy.orm import Session
from .models import User, SessionLocal

app = FastAPI()

def get_db():
    with SessionLocal() as db:  # Контекстный менеджер автоматически закроет сессию
        yield db  # Передаем сессию в зависимость


@app.post('/user/')
def create_user(user: User, db: Session = Depends(get_db)):
    db.add(user)
    db.commit()  # Фиксируем изменения
    db.refresh(user)  # Загружаем обновленные данные из БД
    return user


@app.get('/user/{user_id}')
def get_user(user_id: int, db: Session = Depends(get_db)):
    user = db.query(User).filter(User.id == user_id).first()
    if not user:
        raise HTTPException(status_code=404, detail="User not found")
    return user


### извлекаем данные из бд 

In [ ]:
for user in (
    session.query(User)
    .filter(User.name == '')
    .filter(User.age == 18)
    .limit(2)
    .all()
):
    print(user.id)

### схемы для валидации 
отдельный файл с pydentic

### миграция таблиц alembic